# Traffic Managment System with OpenCV

### Abstract
Control traffic lights based on the vehicle density at four _regions of interest_.
Live video stream obtained from [here](https://www.insecam.org/en/view/751910/).


In [ ]:
import cv2
from urllib.request import urlopen
import numpy as np


#### Video Stream

<img src="test/img/template.jpg" alt="traffic intersection" width="600"/>


In [2]:
#https://kevinsaye.wordpress.com/2018/10/17/making-a-rtsp-server-out-of-a-raspberry-pi-in-15-minutes-or-less/
http://0.0.0.0:5000/video_feed"cap = cv2.VideoCapture("http://0.0.0.0:5000/video_feed") #SNC-CH110 - sony network camera

In [16]:
import cv2
from urllib.request import urlopen
import numpy as np

stream_url = "http://0.0.0.0:5000/video_feed"

with urlopen(stream_url) as stream:
    
    bytes = b''
    while True:
        bytes += stream.read(64*1024)
        a = bytes.find(b'\xff\xd8') #start bytes of mjpeg
        b = bytes.find(b'\xff\xd9') #end bytes of mjpeg
        if a != -1 and b != -1:     #detect start and end pos
            jpg = bytes[a:b+2]      #extract jpeg
            bytes = bytes[b+2:]     #next jpeg item
            img = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8), cv2.IMREAD_GRAYSCALE) #decode jpg raw bytes to np array
#           gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            cv2.imshow("img", img)
#             cv2.imshow("gray", gray)
           
        if cv2.waitKey(1) == 27: #wait for <ESC>
            stream.close()
            break

    cv2.destroyAllWindows()


/home/sj/Code/2019/KITS/Traffic-Management-System/test/speed-detector/env/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



#### Background Extraction

Calculate the weighted sum of the input image src and the accumulator dst so that dst becomes a running average of the frame sequence. [link](https://docs.opencv.org/2.4/modules/imgproc/doc/motion_analysis_and_object_tracking.html?highlight=accumulate#accumulateweighted)

<img src="https://docs.opencv.org/2.4/_images/math/7480f2f9ee402e9e85823d2644b8e1f8c263191a.png" alt="eqn" width="600"/>


In [2]:
# cap = cv2.VideoCapture(1) #in case video stream is inaccessible
cap = cv2.VideoCapture("http://0.0.0.0:5000/video_feed")
alpha = 0.01
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #src
avg = np.float32(gray) #dst

while ret:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.accumulateWeighted(gray, avg, alpha) #running avg calculation
    res_avg = cv2.convertScaleAbs(avg)
    
    cv2.imwrite("test/img/average.jpg", res_avg)
    cv2.imshow("accumulateWeighted", res_avg)
    cv2.imshow("input", frame)
    ret, frame = cap.read()
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#cv2.imwrite("average.jpg", res_avg)
cap.release()
cv2.destroyAllWindows()


Average Image
<img src="test/img/average.jpg" alt="Running verage" width="600"/>

**ISSUE:** Incomplete background extrcation using the above method.

**Possible Soln. :** 

- Changing alpha value. 

- Calculate running avg. for a larger number of frames.

---


#### Masks
Created in gimp

MASK 1|  MASK 2|MASK 3|MASK 4
----|-------|---------|---------
![](test/img/mask_roi_1.jpg) | ![](test/img/mask_roi_2.jpg) | ![](test/img/mask_roi_3.jpg) | ![](test/img/mask_roi_4.jpg)


#### Template Creation
with masks and average frame [cv2.bitwise_and()](https://docs.opencv.org/2.4.8/modules/core/doc/operations_on_arrays.html#bitwise-and)





In [9]:

mask_files = ['test/img/mask_roi_1.jpg',
              'test/img/mask_roi_2.jpg',
              'test/img/mask_roi_3.jpg',
              'test/img/mask_roi_4.jpg']

template_files = ['test/img/template_roi_1.jpg',
                  'test/img/template_roi_2.jpg',
                  'test/img/template_roi_3.jpg',
                  'test/img/template_roi_4.jpg']

crop = [{"x1": 970, "y1": 400, "x2":1279, "y2":500},
        {"x1": 755, "y1": 275, "x2":940, "y2":365},
        {"x1": 335, "y1": 335, "x2":528, "y2":400},
        {"x1": 300, "y1": 470, "x2":670, "y2":675}]


masks = [cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE) for mask_file in mask_files]
average = cv2.imread("test/img/average.jpg", cv2.IMREAD_GRAYSCALE)


for index, (name, mask) in enumerate(zip(template_files, masks)):  
    
    template = cv2.bitwise_and(average, average, mask=mask) ####creating template
    
    x1 = crop[index]["x1"]
    y1 = crop[index]["y1"]
    x2 = crop[index]["x2"]
    y2 = crop[index]["y2"]
    
    template = template[y1:y2, x1:x2]
#     cv2.imwrite(name, template)
    cv2.imshow(name, template)

cv2.waitKey(0)
cv2.destroyAllWindows()    

Template 1|  Template 2|Template 3|Template 4
----|-------|---------|---------
![](test/img/template_roi_1.jpg) | ![](test/img/template_roi_2.jpg) | ![](test/img/template_roi_3.jpg) | ![](test/img/template_roi_4.jpg)


---


#### ROI Extraction 
with masks and frame [cv2.bitwise_and()](https://docs.opencv.org/2.4.8/modules/core/doc/operations_on_arrays.html#bitwise-and)




In [11]:
cap = cv2.VideoCapture("test/vid/h264-night.mp4") #in case video stream is inaccessible
mask_files = ['test/img/mask_roi_1.jpg',
              'test/img/mask_roi_2.jpg',
              'test/img/mask_roi_3.jpg',
              'test/img/mask_roi_4.jpg']
masks = [cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE) for mask_file in mask_files]

crop = [{"x1": 970, "y1": 400, "x2":1279, "y2":500},
        {"x1": 755, "y1": 275, "x2":940, "y2":365},
        {"x1": 335, "y1": 335, "x2":528, "y2":400},
        {"x1": 300, "y1": 470, "x2":670, "y2":675}]

ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

while ret:
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow("input", frame)
    
    for index, (name, mask) in enumerate(zip(mask_files, masks)):  
        res = cv2.bitwise_and(gray, gray, mask=mask)
        x1 = crop[index]["x1"]
        y1 = crop[index]["y1"]
        x2 = crop[index]["x2"]
        y2 = crop[index]["y2"]
        

        res = res[y1:y2, x1:x2] #croppin
#         cv2.imwrite("test/img/roi_{}.jpg".format(index+1), res)
    
        cv2.imshow(name, res)
    
    ret, frame = cap.read()
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



ROI 1|  ROI 2|ROI 3|ROI 4
----|-------|---------|---------
![](test/img/roi_1.jpg) | ![](test/img/roi_2.jpg) | ![](test/img/roi_3.jpg) | ![](test/img/roi_4.jpg)





#### Template Matching
[link](https://docs.opencv.org/2.4.8/modules/imgproc/doc/object_detection.html?highlight=matchtemplate#cv2.matchTemplate)

In [3]:
template_files = ['test/img/template_roi_1.jpg',
                  'test/img/template_roi_2.jpg',
                  'test/img/template_roi_3.jpg',
                  'test/img/template_roi_4.jpg']
# cap = cv2.VideoCapture("rtsp://93.157.18.93/media/video1") #SNC-CH110 - sony network camera
cap = cv2.VideoCapture("test/vid/h264-night.mp4") #in case video stream is inaccessible
templates = [cv2.imread(template_file, cv2.IMREAD_GRAYSCALE) for template_file in template_files]

# mask_files = ["mask_roi_4.jpg"]
mask_files = ['test/img/mask_roi_1.jpg',
              'test/img/mask_roi_2.jpg',
              'test/img/mask_roi_3.jpg',
              'test/img/mask_roi_4.jpg']
masks = [cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE) for mask_file in mask_files]

crop = [{"x1": 970, "y1": 400, "x2":1279, "y2":500},
        {"x1": 755, "y1": 275, "x2":940, "y2":365},
        {"x1": 335, "y1": 335, "x2":528, "y2":400},
        {"x1": 300, "y1": 470, "x2":670, "y2":675}]


ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


while ret:
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("input", frame)
    font = cv2.FONT_HERSHEY_SIMPLEX
    for index, (mask, template) in enumerate(zip(masks, templates)):
        roi = cv2.bitwise_and(gray, gray, mask=mask)
        x1 = crop[index]["x1"]
        y1 = crop[index]["y1"]
        x2 = crop[index]["x2"]
        y2 = crop[index]["y2"]
        

        roi = roi[y1:y2, x1:x2] 
        res = cv2.matchTemplate(roi, template, cv2.TM_CCORR_NORMED)
#       res = cv2.subtract(roi, template)
#       nozero = cv2.countNonZero(res)
        cv2.putText(roi,"{:.3f}".format(res[0][0]),(10,50), font, 1,(255,255,255),2,cv2.LINE_AA)
        
#       cv2.imshow("Template {}".format(index), template)
        cv2.imshow("Roi {}".format(index), roi)
        cv2.imwrite("test/img/roi_density_{}.jpg".format(index+1), roi)
    
        

    ret, frame = cap.read()
            
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ROI 1|  ROI 2|ROI 3|ROI 4
----|-------|---------|---------
![](test/img/roi_density_1.jpg) | ![](test/img/roi_density_2.jpg) | ![](test/img/roi_density_3.jpg) | ![](test/img/roi_density_4.jpg)

